The dataset is used for this competition is synthetic, but based on a real dataset and generated using a CTGAN. The original dataset deals with predicting the biological response of molecules given various chemical properties. Although the features are anonymized, they have properties relating to real-world features.

Import

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

Load

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Read

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/test.csv")
submission = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/sample_submission.csv")

In [ ]:
train

In [ ]:
test

In [ ]:
submission

Shorten train

In [ ]:
short_train = train[: 250000]
short_train

Analyse

In [ ]:
import seaborn as sns

sns.histplot(short_train.target)

Combine

In [ ]:
target = short_train.target

combi = short_train.drop(['target'], axis=1).append(test)
combi

Analyse combi

In [ ]:
combi.info()

Check for null values

In [ ]:
combi.isnull().sum().sum()

Define X and y

In [ ]:
y = target
X = combi[: len(short_train)]
X_test = combi[len(short_train):]

In [ ]:
X.shape, X_test.shape, y.shape

Normalise

In [ ]:
X = (X - X.min()) / (X.max() - X.min())
X

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

selector = SelectKBest(chi2, k=50)

X = selector.fit_transform(X, y)
X_test = selector.transform(X_test)
y.shape, X.shape, X_test.shape

Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=1, shuffle=True)
X_train.shape, X_val.shape, y_train.shape,y_val.shape, X_test.shape

Select Model

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

model = HistGradientBoostingClassifier(max_iter=20000, random_state=1).fit(X_train, y_train)
print(model.score(X_train, y_train))

Predict validation set

In [ ]:
y_pred = model.predict(X_val)
print(model.score(X_val, y_val))

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_val, y_pred))

Predict on test set

In [ ]:
prediction = model.predict(X_test)
#prediction[prediction < 0] = 0
prediction.shape

In [ ]:
submission.target = prediction
submission

In [ ]:
submission.to_csv('submission.csv', index=False)
submission = pd.read_csv("submission.csv")
submission